**SISTEM IDENTIFIKASI *DATA* RUMAH SAKIT**

Sistem ini dibangun berdasarkan 3 data invoice rumah sakit berbeda yang juga memiliki format data berbeda-beda. Tujuan dari sistem ini dibangun adalah untuk menyatukan ketiga data invoice tersebut ke dalam satu format data invoice yang sama.


Cara kerja sistem adalah dengan mengidentifikasi setiap element dari masing-masing kolom yang selanjutnya element tersebut ditentukan termasuk kedalam kolom target mana. Identifikasi terbagi menjadi dua berdasarkan tipe data: String dan Integer.

In [10]:
pip install dateparser

     |████████████████████████████████| 368kB 2.8MB/s 


In [11]:
#import library

import numpy as np
from itertools import combinations
import pandas as pd
from google.colab import files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes
import datetime
import dateparser
import math

**Identifikasi tipe data string**

In [12]:
#Fungsi untuk identifikasi nama pasien dan nama layanan (menggunakan text classification)

def klase(train_x, train_y, data):
    #using tf idf untuk convert data string menjadi angka
    train_x.append(data)
    vect = TfidfVectorizer()
    X = vect.fit_transform(train_x)
    
    X_test = X[-1]
    X_train = X[:-1]

    #Klasifikasi menggunakan Naive Bayes
    # fit data training ke dalam classifier
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(X_train,train_y)
    # predict data tes
    predict = Naive.predict(X_test)
    #result.append(predict[0])
    train_y.append(predict[0])
  
    #cek hasil
    cek = all(x == 0 for x in predict)
    
    if cek == True:
      name = 'Nama Pasien'
    elif cek == False:
      name = 'Nama Layanan'

    return name


In [13]:
#fungsi cek nama dokter

def cek_namadokter(data):
  hasil = True
  cek = False
  for n in range(len(data)):
    if (data[n:n+3] == 'dr.'):
        cek = True
  if cek != True :
    hasil = False
  return hasil

In [14]:
#fungsi cek tanggal 
#menggunakan parsing tanggal untuk memeriksa format tanggal

def cek_tanggal(data):
  import datetime
  import dateparser

  hasil = True
  x = dateparser.parse(data, languages=['id'])
  if type(x) != datetime.datetime:
      hasil = False
  return hasil

**Identifikasi tipe data Integer/float**

In [15]:
#fungsi cek jumlah
def cek_jumlah(x):
  hasil = False
  if (x%1000 != 0):
      #df_target['Jumlah'][i] = arr[i]
    hasil = True
  return hasil
  

In [16]:
#fungsi cek nomor invoice
def cek_nomor(x):
  hasil = False
  mod = x%100
  if (mod != 0):
    hasil = True
  return hasil
  

In [17]:
#cek korelasi penjumlahan antar kolom integer

def total(col1,col2,col3,file,j):
    a = float('NaN')
    b = float('NaN')
    x = float('NaN')
    if (file.iloc[j, col1] + file.iloc[j, col2] == file.iloc[j, col3]):
        a = col1
        b = col2
        x = col3
    elif (file.iloc[j, col2] + file.iloc[j, col3] == file.iloc[j, col1]):
        a = col2
        b = col3
        x = col1
    elif (file.iloc[j, col1] + file.iloc[j, col3] == file.iloc[j, col2]):
        a = col1
        b = col3
        x = col2

    arr = [a,b,x]
    return arr

In [18]:
#cek maximum value
def max(df, i, col_int):
  max = df.iloc[i, col_int[0]]
  for j in col_int:
      if df.iloc[i, j] > max:
          max = j
  return max

**Fungsi umum**

In [19]:
#Fungsi untuk menginisiasi kolom hasil
def init(df):
  arr_name_col = []
  for i in range(df.shape[1]):
    arr_name_col.append([])
    for n in range(2):
      arr_name_col[i].append('noname')

  return arr_name_col

In [20]:
#Fungsi utama identifikasi data 
def clf_result(i, train_klas_x, train_klas_y, col_result, data):
    for j in range(data.shape[1]):
      #arr_col.append(j)
      if (pd.isna(data.iloc[i, j]) == False):
        #cek identifikasi tipe data string
        if (type(data.iloc[i, j])==str):
            x = data.iloc[i, j]
            
            if (cek_namadokter(x)):
                name = 'Nama Dokter'
                in_arr = []
                in_arr.append(j)
                in_arr.append(name)
                col_result.append(in_arr)
            elif (cek_tanggal(x)):
                name = 'Tanggal'
                in_arr = []
                in_arr.append(j)
                in_arr.append(name)
                col_result.append(in_arr)
                #klas(data, name)
            else:
                name = klase(train_klas_x, train_klas_y, x)
                in_arr = []
                in_arr.append(j)
                in_arr.append(name)
                col_result.append(in_arr)

    col_int = []
    for j in range(data.shape[1]):
      if (pd.isna(data.iloc[i, j]) == False):
        if (type(data.iloc[i, j])!=str):
            col_int.append(j)

    print(col_int)
    maks = max(data, i, col_int)

    #cek tipe data integer
    while (True):
      arr = []
      arred = []
      for combo in combinations(col_int, 3):  # 2 for pairs, 3 for triplets, etc
          arr = total(combo[0],combo[1],combo[2],data, i)
          if not(all(math.isnan(x) for x in arr)):
            arred = arr
      #print(arred)
      if not(all(math.isnan(x) for x in arred)):
        col_int.remove(arred[2])
      else:
        break
      arr2 = []
      arred2 = []
      for combo in combinations(col_int, 3):  # 2 for pairs, 3 for triplets, etc
          arr2 = total(combo[0],combo[1],combo[2],data, i)
          if not(all(math.isnan(x) for x in arr2)):
            arred2 = arr2
      #print(arred2)
      if not(all(math.isnan(x) for x in arred2)):
        col_int.remove(arred2[2])
      else:
        if (data.iloc[i, arred[0]] > data.iloc[i, arred[1]]):
          in_arr = []
          in_arr.append(arred[0])
          in_arr.append('Harga Satuan')
          col_result.append(in_arr)
        elif (data.iloc[i, arred[0]] < data.iloc[i, arred[1]]):
          in_arr = []
          in_arr.append(arred[1])
          in_arr.append('Harga Satuan')
          col_result.append(in_arr)

        if arred[2] == maks:
          in_arr = []
          in_arr.append(arred[2])
          in_arr.append('Harga Total')
          col_result.append(in_arr)
        break
      if all(math.isnan(x) for x in arred) or all(math.isnan(x) for x in arred2):
        break
      else:
        cek1 = any(x == arred[2] for x in arred2)
        cek2 = any(x == arred2[2] for x in arred)
        if cek1 == True:
          #print(arred[2])
          print('harga 1 = '+ str(arred[0]))
          print('harga 2 = '+ str(arred[1]))
          in_arr = []
          in_arr.append(arred[0])
          in_arr.append('Harga Satuan')
          col_result.append(in_arr)
          #append another harga satuan
          in_arr = []
          in_arr.append(arred[1])
          in_arr.append('Harga Satuan')
          col_result.append(in_arr)

          if arred2[2] == maks:
            print('harga total = '+ str(arred2[2]))
            #col_result[arred2[2]] = 'Harga Total'
            in_arr = []
            in_arr.append(arred2[2])
            in_arr.append('Harga Total')
            col_result.append(in_arr)
          break
        elif cek2 == True:
          #print(arred2[2])
          print('harga 1 = '+ str(arred2[0]))
          print('harga 2 = '+ str(arred2[1]))
          in_arr = []
          in_arr.append(arred2[0])
          in_arr.append('Harga Satuan')
          col_result.append(in_arr)
          #append another harga satuan
          in_arr = []
          in_arr.append(arred2[1])
          in_arr.append('Harga Satuan')
          col_result.append(in_arr)
          
          if arred[2] == maks:
            print('harga total = ' + str(arred[2]))
            #col_result[arred[2]] = 'Harga Total'
            in_arr = []
            in_arr.append(arred[2])
            in_arr.append('Harga Total')
            col_result.append(in_arr)
          break
    #print('loop ended') 
    if (all(math.isnan(x) for x in arred)):
      for col in col_int:
        x = data.iloc[i, col]
        if cek_nomor(x):
          print('nomor')
          #col_int.remove(col)
        elif cek_jumlah(x):
          print('jumlah')
          #col_int.remove(col)
        elif col == maks:
          in_arr = []
          in_arr.append(col)
          in_arr.append('Harga Total')
          col_result.append(in_arr)
        else:
          in_arr = []
          in_arr.append(col)
          in_arr.append('Harga Satuan')
          col_result.append(in_arr)
    cek_hrgstn = False
    num_hrgt = 0
    for p in range(len(col_result)):
      if col_result[p][1] == 'Harga Satuan':
        cek_hrgstn = True 
      if col_result[p][1] == 'Harga Total':
        num_hrgt = col_result[p][0]
    if cek_hrgstn == False:
      in_arr = []
      in_arr.append(num_hrgt)
      in_arr.append('Harga Satuan')
      col_result.append(in_arr)
    print(col_result)



In [21]:
#Fungsi untuk memasukan element data ke dalam kolom target yang sesuai
def to_target(i, col_result, data, df_target):
  jml_lyn = 0
  for j in range(len(col_result)):
    n = col_result[j].count('Nama Layanan')
    jml_lyn = jml_lyn+n 

  jml_psn = 0
  for j in range(len(col_result)):
    n = col_result[j].count('Nama Pasien')
    jml_psn = jml_psn+n 

  while (jml_lyn >= 1 and jml_psn >= 1):
    arr = []
    for n in range(len(df_target.columns)):
      for k in range(len(col_result)):
        if (df_target.columns[n] == col_result[k][1]):
          break 
      if (df_target.columns[n] == col_result[k][1]):
        if (df_target.columns[n] == 'Nama Layanan' or df_target.columns[n] == 'Harga Satuan'):
          col_result[k][1] = 'noname'
        arr.append(data.iloc[i, col_result[k][0]])
      #arr.append(float('NaN'))    
    print(arr)
    df_target.loc[df_target.shape[0]+1] = arr
    jml_lyn -= 1

In [22]:
#Data latih untuk text classification, identifikasi nama pasien dan nama layanan
arr_x = ['Kamaria Astuti S.E.I', 'Raditya Natsir', 'Jono Jaeman Wibisono', 'Kamila Anggraini', 'Sari Rahmawati', 'Yessi Astuti', 'Dartono Manullang', 'Sabrina Prastuti S.T.', 'Genta Suci Mandasari S.IP', 'Nyana Maheswara', 'Jaswadi Nrima Maheswara S.Farm', 'Anastasia Pertiwi', 'Umi Humaira Pudjiastuti S.I.Kom', 'Nilam Novitasari S.I.Kom', 'Cawisadi Prayoga M.Kom.', 'Gada Winarno', 'Nyana Waskita', 'Gatra Najmudin', 'Talia Mandasari S.Ked', 'Mitra Prasasta', 'Ridwan Samosir', 'Bakijan Radit Simbolon S.Farm', 'Cornelia Ratih Hariyah S.E.', 'Cici Safitri S.IP', 'Nurul Tina Hartati', 'Cakrabirawa Lanang Maulana S.Ked', 'Putri Ayu Namaga', 'Kasiran Kusuma Siregar', 'Oskar Gangsa Gunarto', 'Farhunnisa Mardhiyah', 'Bella Eli Wulandari S.IP', 'Kayla Susanti', 'Galuh Dongoran', 'Eka Damanik', 'Tami Titi Lailasari', 'Candra Mangunsong S.Gz', 'Paulin Suci Mardhiyah', 'Atma Simbolon S.Kom', 'Yusuf Saptono', 'Fathonah Oktaviani', 'Ophelia Queen Riyanti M.M.', 'Rudi Wibowo', 'Hartaka Sitompul S.Psi', 'Hairyanto Marbun', 'Mulya Mandala S.I.Kom', 'Fathonah Samiah Wastuti', 'Daniswara Prakasa S.IP', 'Rizki Ivan Sinaga M.Ak', 'Yessi Zamira Novitasari', 'Ian Xanana Firgantoro S.Gz', 'Latika Puti Agustina', 'Oskar Natsir', 'Puji Padmasari S.Farm', 'Karman Prasasta', 'Ganda Mansur M.Farm', 'Jais Suryono', 'Reksa Permadi', 'Faizah Agustina', 'Talia Puspita', 'Putri Azalea Nurdiyanti', 'Ratih Wastuti', 'Emong Wijaya M.TI.', 'Mumpuni Marbun', 'Hasna Zulfa Rahimah', 'Irma Susanti', 'Amalia Zulaika', 'Asmianto Irawan S.Ked', 'Usyi Hassanah S.Pt', 'Luthfi Jono Setiawan S.Gz', 'Yahya Waluyo', 'Ella Pratiwi', 'Bahuwarna Megantara', 'Gaman Suryono', 'Puput Sabrina Pudjiastuti', 'Aditya Januar', 'Xanana Thamrin', 'Koko Okto Mandala M.Kom.', 'Ulva Suryatmi', 'Ayu Riyanti S.Kom', 'Icha Hariyah', 'Teddy Saefullah', 'Unggul Zulkarnain', 'Daru Jaeman Hidayat', 'Nilam Calista Riyanti', 'Eli Puspa Hariyah S.Farm', 'Gamanto Garang Salahudin M.Kom.', 'Ikin Hakim', 'Kajen Napitupulu S.Psi', 'Kajen Manullang', 'Asmadi Asirwada Haryanto', 'Legawa Firmansyah', 'Rendy Marpaung M.Kom.', 'Restu Kuswandari', 'Kemba Arta Wijaya S.Farm', 'Kajen Ramadan', 'Kasusra Wahyudin', 'Fitriani Yolanda', 'Cemplunk Hutagalung', 'Jagaraga Lazuardi', 'Hamima Zizi Widiastuti M.Farm', 'Akupuntur', 'Klinik Anak', 'Andrologi', 'Anesthesi', 'Bedah Jantung, Thorax, dan Pembuluh Darah', 'Bedah Plastik', 'Bedah Syaraf', 'Orthopedi dan Traumatologi', 'Onkologi', 'Bedah Umum', 'Bedah Urologi', 'Gigi dan Mulut', 'Klinik Gizi', 'Klinik Jiwa', 'Kardiologi', 'Kebidanan dan Kandungan', 'Kulit dan Kelamin', 'Mata', 'Paru', 'Penyakit Dalam', 'Syaraf', 'Telinga, Hidung dan Tenggorokan (THT)', 'Okupasi', 'Instalasi Gawat Darurat (IGD)', 'Instalas Medical Check-Up', 'Rehabilitasi Medik', 'Intensive Care Unit (ICU)', 'Stroke Unit', 'Hemodialisa', 'Unit Luka Bakar', 'Akupuntur', 'Klinik Anak', 'Andrologi', 'Anesthesi', 'Bedah Jantung, Thorax, dan Pembuluh Darah', 'Bedah Plastik', 'Bedah Syaraf', 'Orthopedi dan Traumatologi', 'Onkologi', 'Bedah Umum', 'Bedah Urologi', 'Gigi dan Mulut', 'Klinik Gizi', 'Klinik Jiwa', 'Kardiologi', 'Kebidanan dan Kandungan', 'Kulit dan Kelamin', 'Mata', 'Paru', 'Penyakit Dalam', 'Syaraf', 'Telinga, Hidung dan Tenggorokan (THT)', 'Okupasi', 'Intensive Care Unit (ICU)', 'Instalasi Gawat Darurat (IGD)', 'Instalas Medical Check-Up', 'Rehabilitasi Medik', 'Intensive Care Unit (ICU)', 'Stroke Unit', 'Hemodialisa', 'Unit Luka Bakar', 'Akupuntur', 'Klinik Anak', 'Andrologi', 'Anesthesi', 'Bedah Jantung, Thorax, dan Pembuluh Darah', 'Bedah Plastik', 'Bedah Syaraf', 'Orthopedi dan Traumatologi', 'Onkologi', 'Bedah Umum', 'Bedah Urologi', 'Gigi dan Mulut', 'Klinik Gizi', 'Klinik Jiwa', 'Kardiologi', 'Kebidanan dan Kandungan', 'Kulit dan Kelamin', 'Mata', 'Paru', 'Penyakit Dalam', 'Syaraf', 'Telinga, Hidung dan Tenggorokan (THT)', 'Okupasi', 'Konsultasi Dokter Umum', 'Konsultasi Dokter Umum', 'Konsultasi Dokter Umum', 'Klinik Anak', 'Andrologi', 'Anesthesi', 'Bedah Jantung, Thorax, dan Pembuluh Darah', 'Bedah Plastik', 'Bedah Syaraf', 'Orthopedi dan Traumatologi', 'Onkologi', 'Bedah Umum', 'Bedah Urologi', 'Gigi dan Mulut', 'Telinga, Hidung dan Tenggorokan (THT)', 'Okupasi']

arr_y = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [23]:
#Membuat DataFrame untuk Target kolom

df_target = pd.DataFrame(columns=['Tanggal', 'Nama Pasien', 'Nama Layanan', 'Harga Satuan', 'Harga Total'])
df_target

,Tanggal,Nama Pasien,Nama Layanan,Harga Satuan,Harga Total


In [24]:
#Fungsi Utama

print('-----------------------------------------------------------------------')
print('---------------- SISTEM DATA INVOICE RUMAH SAKIT ----------------------')
print('-----------------------------------------------------------------------')
print('\n')


itr = True
while (itr):
    print('Masukkan data invoice Rumah Sakit')
    files.upload()

    x = input('Masukkan nama file = ')
    data = pd.read_csv(x)

    col_result = []
    arr_klas = []
    arr_klas_result = []
    for i in range(data.shape[0]):
      col_result = []
      clf_result(i, arr_x, arr_y, col_result, data)
      to_target(i, col_result, data, df_target)

    ans = input('Apakah ingin memasukkan data invoice lagi? (ya/tidak)')
    if ans.lower() == 'tidak' :
       itr = False

print('\n')
df_target

-----------------------------------------------------------------------
---------------- SISTEM DATA INVOICE RUMAH SAKIT ----------------------
-----------------------------------------------------------------------


Masukkan data invoice Rumah Sakit


Saving RS_1.csv to RS_1.csv
Masukkan nama file = RS_1.csv
[3, 5, 6, 7, 8]
harga 1 = 3
harga 2 = 5
harga total = 8
[[0, 'Tanggal'], [1, 'Nama Pasien'], [2, 'Nama Layanan'], [4, 'Nama Layanan'], [3, 'Harga Satuan'], [5, 'Harga Satuan'], [8, 'Harga Total']]
['13/11/2020', 'Sari Zelda Purwanti M.M.', 'Instalasi Gawat Darurat (IGD)', 2000000, 2110000]
['13/11/2020', 'Sari Zelda Purwanti M.M.', 'Perawatan Umum', 100000.0, 2110000]
[3, 5, 6, 7, 8]
harga 1 = 3
harga 2 = 5
harga total = 8
[[0, 'Tanggal'], [1, 'Nama Pasien'], [2, 'Nama Layanan'], [4, 'Nama Layanan'], [3, 'Harga Satuan'], [5, 'Harga Satuan'], [8, 'Harga Total']]
['3/11/2020', 'Darimin Kusumo S.Ked', 'Instalas Medical Check-Up', 200000, 360000]
['3/11/2020', 'Darimin Kusumo S.Ked', 'Perawatan Khusus', 150000.0, 360000]
[3, 5, 6, 7, 8]
harga 1 = 3
harga 2 = 5
harga total = 8
[[0, 'Tanggal'], [1, 'Nama Pasien'], [2, 'Nama Layanan'], [4, 'Nama Layanan'], [3, 'Harga Satuan'], [5, 'Harga Satuan'], [8, 'Harga Total']]
['25/12/2020', 'Sa

Saving RS_2.csv to RS_2.csv
Masukkan nama file = RS_2.csv
[0, 4, 5, 6]
[[1, 'Nama Pasien'], [2, 'Nama Dokter'], [3, 'Nama Layanan'], [7, 'Tanggal'], [4, 'Harga Satuan'], [6, 'Harga Total']]
['04/04/2020', 'Uli Mandasari', 'Penyakit dalam', 800000, 840000]
[0, 4, 5, 6]
[[1, 'Nama Layanan'], [2, 'Nama Dokter'], [3, 'Nama Layanan'], [7, 'Tanggal'], [4, 'Harga Satuan'], [6, 'Harga Total']]
[0, 4, 5, 6]
[[1, 'Nama Pasien'], [2, 'Nama Dokter'], [3, 'Nama Layanan'], [7, 'Tanggal'], [4, 'Harga Satuan'], [6, 'Harga Total']]
['02/06/2020', 'Usyi Zaenab Suartini M.TI.', 'Penyakit syaraf ', 2000000, 2100000]
[0, 4, 5, 6]
[[1, 'Nama Layanan'], [2, 'Nama Dokter'], [3, 'Nama Layanan'], [7, 'Tanggal'], [4, 'Harga Satuan'], [6, 'Harga Total']]
[0, 4, 5, 6]
[[1, 'Nama Layanan'], [2, 'Nama Dokter'], [3, 'Nama Layanan'], [7, 'Tanggal'], [4, 'Harga Satuan'], [6, 'Harga Total']]
[0, 4, 5, 6]
[[1, 'Nama Layanan'], [2, 'Nama Dokter'], [3, 'Nama Layanan'], [7, 'Tanggal'], [4, 'Harga Satuan'], [6, 'Harga Total'

Saving RS_3.csv to RS_3.csv
Masukkan nama file = RS_3.csv
[0, 5]
nomor
[[1, 'Tanggal'], [2, 'Nama Pasien'], [3, 'Nama Dokter'], [4, 'Nama Layanan'], [5, 'Harga Total'], [5, 'Harga Satuan']]
['3/8/2020', 'Chandra Wasita', 'Pemeriksaan Dokter', 150000, 150000]
[0, 5]
nomor
[[1, 'Tanggal'], [2, 'Nama Pasien'], [3, 'Nama Dokter'], [4, 'Nama Layanan'], [5, 'Harga Total'], [5, 'Harga Satuan']]
['21/12/2020', 'Cici Nurdiyanti', 'Rawat Jalan', 131000, 131000]
[0, 5]
nomor
[[1, 'Tanggal'], [2, 'Nama Pasien'], [3, 'Nama Dokter'], [4, 'Nama Layanan'], [5, 'Harga Total'], [5, 'Harga Satuan']]
['18/2/2020', 'Hasta Eko Mansur S.I.Kom', 'Farmasi', 739000, 739000]
[0, 5]
nomor
[[1, 'Tanggal'], [2, 'Nama Pasien'], [3, 'Nama Dokter'], [4, 'Nama Layanan'], [5, 'Harga Total'], [5, 'Harga Satuan']]
['20/4/2020', 'Malika Belinda Wastuti S.Gz', 'Pemeriksaan Dokter', 150000, 150000]
[0, 5]
nomor
[[1, 'Tanggal'], [2, 'Nama Layanan'], [3, 'Nama Dokter'], [4, 'Nama Layanan'], [5, 'Harga Total'], [5, 'Harga Satua

,Tanggal,Nama Pasien,Nama Layanan,Harga Satuan,Harga Total
1,13/11/2020,Sari Zelda Purwanti M.M.,Instalasi Gawat Darurat (IGD),2000000,2110000
2,13/11/2020,Sari Zelda Purwanti M.M.,Perawatan Umum,100000,2110000
3,3/11/2020,Darimin Kusumo S.Ked,Instalas Medical Check-Up,200000,360000
4,3/11/2020,Darimin Kusumo S.Ked,Perawatan Khusus,150000,360000
5,25/12/2020,Saiful Sitompul,Rehabilitasi Medik,240000,380000
...,...,...,...,...,...
254,11/6/2020,Wirda Nasyiah S.IP,Pelayanan Konseling pastoral,313000,313000
255,21/2/2020,Malik Cawisono Winarno M.Farm,Farmasi,739000,739000
256,15/10/2020,Yunita Nasyiah,Perawatan Khusus,65000,65000
257,17/5/2020,Rahman Utama,Pelayanan Konseling pastoral,313000,313000
